<div style="max-width:1200px"><img src="../_resources/mgnify_banner.png" width="100%"></div>

<img src="../_resources/mgnify_logo.png" width="200px">

# Fetch a Study using MGnifyR; download the metadata for all of its Analyses

The [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1) returns data and relationships as JSON. 
[MGnifyR](https://github.com/beadyallen/MGnifyR) is a package to help you read MGnify data into your R analyses.

**This example shows you how fetch the Analyses for a Study into a Phyloseq object**

You can find all of the other "API endpoints" using the [Browsable API interface in your web browser](https://www.ebi.ac.uk/metagenomics/api/v1).

This is an interactive code notebook (a Jupyter Notebook).
To run this code, click into each cell and press the ▶ button in the top toolbar, or press `shift+enter`.

---

In [ ]:
source("lib/variable_utils.r")

In [ ]:
mgnify_study_accession <- get_variable_from_link_or_input('MGYS', 'Study Accession', 'MGYS00005116')

# You can also just directly set the accession variable in code, like this:
# mgnify_study_accession <- "MGYS00005292"

In [ ]:
suppressMessages({
    library(vegan)
    library(ggplot2)
    library(phyloseq)
    library(MGnifyR)
})

mg <- MgnifyClient(useCache = T, cacheDir = '/home/mgnify/.mgnify_cache')

In [ ]:
library(IRdisplay)
display_markdown(file = '../_resources/mgnifyr_help.md')

## Fetch a list of the Analyses for the Study

In [ ]:
analyses_accessions <- searchAnalysis(mg, "studies", mgnify_study_accession)
analyses_accessions

## Download metadata for the first 10 Analyses
...and put it into a dataframe.

In [ ]:
analyses_metadata_df <- getMetadata(mg, head(analyses_accessions, 10));

## Display metadata
The table could be big, so let's look at a sample of it (`head`)

In [ ]:
t(head(analyses_metadata_df))

## Convert to [TreeSummarizedExperiment](https://bioconductor.org/packages/release/bioc/vignettes/TreeSummarizedExperiment/inst/doc/Introduction_to_treeSummarizedExperiment.html)
> The `TreeSummarizedExperiment` class is ... used to store rectangular data of experimental results ... and also supports the storage of a hierarchical structure and its link information to the rectangular data.

For example, the MGnifyR method `getResult` can be used to fetch a TSE of taxonomic assignments of the rRNA Small SubUnit (SSU):

In [ ]:
analyses_tse <- getResult(mg, analyses_metadata_df$analysis_accession, get.taxa = TRUE, get.func = FALSE, taxa.su = "SSU")

You can also get a  [phyloseq](https://joey711.github.io/phyloseq/) object if you are more familiar with that ecosystem.
> The phyloseq package is a tool to import, store, analyze, and graphically display complex phylogenetic sequencing data that has already been clustered into Operational Taxonomic Units (OTUs), especially when there is associated sample data, phylogenetic tree, and/or taxonomic assignment of the OTUs."

You can then use any of the `phyloseq` methods to explore this Study's Analyses (run `?phyloseq` in a code cell to learn more). 

Example of Phyloseq usage:

In [ ]:
analyses_phylo <- getResult(mg, analyses_metadata_df$analysis_accession, get.taxa = TRUE, get.func = FALSE, taxa.su = "SSU", output="phyloseq")

In [ ]:
options(repr.plot.width=12, repr.plot.height=5)
plot_phyloseq(analyses_ps, )